In [9]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [11]:
dfCrime = pd.read_excel('/Users/austincoffelt/Downloads/NIBRSPublicView2021.xlsx')
print(dfCrime.shape)
dfCrime.head()

(250699, 16)


,Incident,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,NIBRSDescription,OffenseCount,Beat,Premise,StreetNo,StreetName,StreetType,Suffix,City,ZIPCode,MapLongitude,MapLatitude
0,1621,2021-01-01,0,13A,Aggravated Assault,1,18F40,"Residence, Home (Includes Apartment)",8787,WOODWAY,DR,NaN,HOUSTON,77063.0,-95.518310,29.740998
1,1621,2021-01-01,0,13B,Simple assault,1,18F40,"Residence, Home (Includes Apartment)",8787,WOODWAY,DR,NaN,HOUSTON,77063.0,-95.518310,29.740998
2,8821,2021-01-01,0,35A,"Drug, narcotic violations",1,19G30,"Highway, Road, Street, Alley",13339,VERBENA,LN,NaN,HOUSTON,77083.0,-95.617882,29.707451
3,8821,2021-01-01,0,23F,Theft from motor vehicle,1,19G30,"Highway, Road, Street, Alley",13339,VERBENA,LN,NaN,HOUSTON,77083.0,-95.617882,29.707451
4,11721,2021-01-01,0,90C,Disorderly conduct,1,12D10,"Residence, Home (Includes Apartment)",8839,BARTON,ST,NaN,HOUSTON,77075.0,-95.259399,29.642036


In [17]:
violentCrimes = ['Aggravated Assault', 'Forcible fondling', 'Forcible rape', 'Forcible sodomy', 'Murder, non-negligent',
                'Robbery']
dfCrime = dfCrime[dfCrime['NIBRSDescription'].isin(violentCrimes)]
print(dfCrime.shape)
dfCrime.head()

(26624, 16)


,Incident,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,NIBRSDescription,OffenseCount,Beat,Premise,StreetNo,StreetName,StreetType,Suffix,City,ZIPCode,MapLongitude,MapLatitude
0,1621,2021-01-01,0,13A,Aggravated Assault,1,18F40,"Residence, Home (Includes Apartment)",8787,WOODWAY,DR,NaN,HOUSTON,77063.0,-95.518310,29.740998
6,17021,2021-01-01,0,13A,Aggravated Assault,1,19G10,"Residence, Home (Includes Apartment)",10630,BEECHNUT,ST,NaN,HOUSTON,77072.0,-95.567421,29.688969
8,22721,2021-01-01,0,13A,Aggravated Assault,1,1A20,"Highway, Road, Street, Alley",635,WEBSTER,ST,NaN,HOUSTON,77002.0,-95.375132,29.749892
10,26221,2021-01-01,0,13A,Aggravated Assault,1,14D40,"Residence, Home (Includes Apartment)",11911,MARTIN LUTHER KING,BLVD,NaN,HOUSTON,77048.0,-95.339345,29.631254
11,29021,2021-01-01,0,13A,Aggravated Assault,3,14D20,"Residence, Home (Includes Apartment)",7209,LA SALETTE,ST,NaN,HOUSTON,77021.0,-95.367462,29.685922


In [19]:
dfCrime = dfCrime[['NIBRSDescription', 'MapLatitude', 'MapLongitude']]

# make our lon and lat into a point type for spacial join
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['MapLongitude'], row['MapLatitude']), axis=1)
dfCrime = dfCrime[['NIBRSDescription', 'geometry']]
dfCrime.head()

,NIBRSDescription,geometry
0,Aggravated Assault,POINT (-95.51831 29.740998)
6,Aggravated Assault,POINT (-95.567421 29.688969)
8,Aggravated Assault,POINT (-95.37513199999999 29.749892)
10,Aggravated Assault,POINT (-95.33934499999999 29.631254)
11,Aggravated Assault,POINT (-95.367462 29.685922)


In [21]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("cb_2022_us_bg_500k/cb_2022_us_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(242297, 12)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,04,019,940800,2,1500000US040199408002,040199408002,2,Block Group 2,BG,2033911373,68536,"POLYGON ((-112.41 31.857, -112.38 31.859, -112..."
1,04,003,000901,4,1500000US040030009014,040030009014,4,Block Group 4,BG,486316,0,"POLYGON ((-109.55 31.334, -109.55 31.339, -109..."
2,06,073,018102,3,1500000US060730181023,060730181023,3,Block Group 3,BG,324091,3579,"POLYGON ((-117.37 33.182, -117.37 33.182, -117..."
3,06,073,007200,3,1500000US060730072003,060730072003,3,Block Group 3,BG,720442,709604,"POLYGON ((-117.26 32.725, -117.26 32.726, -117..."
4,06,073,017702,3,1500000US060730177023,060730177023,3,Block Group 3,BG,317237,0,"POLYGON ((-117.3 33.052, -117.3 33.053, -117.3..."


In [23]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfCrime, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,NIBRSDescription,geometry
0,Aggravated Assault,POINT (-95.518 29.741)
6,Aggravated Assault,POINT (-95.567 29.689)
8,Aggravated Assault,POINT (-95.375 29.75)
10,Aggravated Assault,POINT (-95.339 29.631)
11,Aggravated Assault,POINT (-95.367 29.686)


In [25]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result.shape)
result.head()

(26624, 14)


,NIBRSDescription,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER
0,Aggravated Assault,POINT (-95.518 29.741),141602.0,48,201,431205,3,1500000US482014312053,482014312053,3,Block Group 3,BG,47568.0,0.0
6,Aggravated Assault,POINT (-95.567 29.689),138987.0,48,201,453100,2,1500000US482014531002,482014531002,2,Block Group 2,BG,242417.0,0.0
8,Aggravated Assault,POINT (-95.375 29.75),140560.0,48,201,410602,1,1500000US482014106021,482014106021,1,Block Group 1,BG,142322.0,0.0
10,Aggravated Assault,POINT (-95.339 29.631),139367.0,48,201,331602,3,1500000US482013316023,482013316023,3,Block Group 3,BG,902775.0,1005.0
11,Aggravated Assault,POINT (-95.367 29.686),202207.0,48,201,313801,2,1500000US482013138012,482013138012,2,Block Group 2,BG,337861.0,0.0


In [27]:
crime = result['AFFGEOID'].value_counts()
crime

AFFGEOID
1500000US482013209012    146
1500000US482013333013    131
1500000US482019807001    119
1500000US482014336023    111
1500000US482013125011    104
                        ... 
1500000US482013242002      1
1500000US482013504002      1
1500000US482014117002      1
1500000US482014514041      1
1500000US482014122023      1
Name: count, Length: 1636, dtype: int64

In [29]:
# export as a csv
crime.to_csv('Houston_Violent_Crime_BG.csv')